# Binomial with Beta Prior

## Motivating Example

Before we dive into the mathematics, let us consider a motivating example:

---

One day while walking downtown, you stop briefly to watch a street-performer - a magician - who is doing various sleight-of-hand tricks. As a part of his next trick, he shows the crowd a standard-appearing coin with both head and tails sides. Somewhat abruptly the magician singles you out from the crowd and offers you a deal: if he flips the coin 10 times and it lands on heads 7 or less times, he will pay you 100 dollars, but if it lands on heads 8 or more times, you owe him 100 dollars. The question: do you take the bet?

---

## Expected Value of A Fair Coin

Let us first calculate the expected value of the above proposition assuming the coin is fair.

To begin we model the coin as a random variable over $n$ Bernoulli trials with $x$ number of heads and $n-x$ tails according to some unknown probability $\theta$ of heads and a complementary probability $(1-\theta)$ of tails.

The random variable will follow the binomial distribution with a probability mass function (pmf) of $$p(n,x|\theta)={n \choose x}\theta^{x}(1-\theta)^{n-x}$$

We begin by graphing the probability of each combination of heads and tails and coloring it by its associated betting outcome, in addition to calculate the expected value of the bet.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import binom

# set parameter values
n = 10 # number of flips
theta = 0.5 # probability of heads

#define possible outcomes of heads
x_values = np.arange(n+1)

#calculate pmf values
dist = np.array([binom.pmf(x,n,theta) for x in x_values])

#create color masks
win = x_values<=7
lose = x_values>7

#plot the distribution
plt.bar(s_values[win],dist[win],color='green',label='Win $100')
plt.bar(s_values[lose],dist[lose],color='red',label='Lose $100')
plt.ylabel('Probability',fontsize='15')
plt.xlabel('Number of Heads',fontsize='15')
plt.legend()
plt.show()

# calculate expected value
expected_value = 100*sum(dist[win]) - 100*sum(dist[lose])
print("Expected Value:",f"${round(expected_value,2)}")

NameError: name 's_values' is not defined

Given the above calculation, the proposition sounds like a wonderful deal with an expected value of $89.06 in your favor if the coin is fair; however, how do you know whether the coin is fair? Better yet, how would you quantify your uncertainty about the fairness of the coin? Suppose, in this hypothetical (although unlikely in reality) that the magician allows you to flip the coin a few times prior to taking the bet. How could you use the data that you collect to inform your decision by updating your uncertainty?

## Parameterizing a Possibly Unfair Coin

As before, we model the coin as a random variable over $n$ Bernoulli trials with $x$ number of heads and $n-x$ number of tails according to some unknown probability $\theta$ of heads and a complementary probability $(1-\theta)$ of tails.

The random variable will follow the binomial distribution with a probability mass function of 

$$p(n,x|\theta)={n \choose x}\theta^{x}(1-\theta)^{n-x}$$

The conjugate prior $p(\theta)$ of the binomial distribution is the beta distribution.

$$p(\theta)=\frac{\theta^{\alpha-1}(1-\theta)^{\beta-1}}{B(\alpha,\beta)}$$

where $\alpha$ and $\beta$ are hyperparameters encoding prior belief or information and $B(\alpha,\beta)$ represents the Beta function serving as a normalizing constant.

Plugging the above equations into Bayes' theorem, simplifying, and renormalizing, 

$$
\begin{align}
p(\theta|n,x) & = \frac{p(n,x|\theta)p(\theta)}{\int p(n,x|\theta)p(\theta)d\theta} \\
& = \frac{{n \choose x}\theta^{x+\alpha-1}(1-\theta)^{n-x+\beta-1}/B(\alpha,\beta)}{\int_{0}^{1} {n \choose x}\theta^{x+\alpha-1}(1-\theta)^{n-x+\beta-1}/B(\alpha,\beta)d\theta} \\
& = \frac{\theta^{x+\alpha-1}(1-\theta)^{n-x+\beta-1}}{B(x+\alpha,n-x+\beta)} \\
\end{align}
$$
which is a Beta distribution with parameters $(x+\alpha,n-x+\beta)$. The posterior distribution can then be used as a prior for further observations with the hyperparameters consolidating the information with each new addition. 

## Intuition of the Beta Parameters

The $\alpha$ and $\beta$ parameters of the Beta prior can seem somewhat cryptic at first, until one recognizes that they represent pseudo-observations. A Beta prior with parameters $(1,1)$ is uninformative as it represents a coin where one has witnessed no coin flips. Meanwhile a Beta prior of $(86,16)$ can be understood as a coin which we have flipped 100 times, with 85 heads and 15 tails. Thus in the process of updating our Beta prior using Bayes' theorem, it makes sense that the resulting posterior distribution would have parameters $(x+\alpha,n-x+\beta)$.


## Estimating the Bias of a Coin

Revisiting the motivating example, suppose you test the coin by flipping it 50 times and it lands on heads exactly 42 times and tails 8 times. Assuming an uninformative prior of $Beta(1,1)$, the posterior would have the form of $Beta(1+42,1+8)=Beta(43,9)$.

In [ ]:
from scipy.stats import beta

# set parameter values
prior_alpha, prior_beta = 1,1
posterior_alpha, posterior_beta = 43, 9

# generate plot values
x = np.linspace(0,1,100)

# generate distributions
prior_dist = beta.pdf(x,prior_alpha,prior_beta)
posterior_dist = beta.pdf(x,posterior_alpha,posterior_beta)

# plot the prior and posterior distributions
plt.plot(x,prior_dist,color='red',label="Prior: Beta(1,1)")
plt.plot(x,posterior_dist,color='blue',label="Posterior: Beta(43,9)")
plt.xlabel('θ',fontsize='15')
plt.ylabel('Probability',fontsize='15')
plt.legend()
plt.show()

From a single glance, we can see that the probability of θ being 0.5 (i.e. the coin being fair) is vanishingly small, while most of the consolidation is occurring somewhere around 0.83.

We can more specifically characterize the posterior distribution by calculating the maximum a posteriori (MAP) which is the global maximum of the posterior distribution. The MAP can provide a good point estimate for the value of theta in distributions that are unimodal and well-behaved.

Further, we can calculate a credible interval for the posterior distribution. A credible interval is the interval over which an unobserved parameter (such as θ) falls with a particular probability. Although there are many ways to define credible intervals for different purposes, for this example we will use a 95% highest density interval, which is defined as the smallest interval on the posterior density which integrates to a given confidence level.

In [ ]:
# calculate maximum a posteriori
p_max = max(posterior_dist)
theta_max = x[np.argmax(posterior_dist)]

#define function to calculate highest density credible interval
from scipy import optimize

def hdi(distribution, level=0.95):
    """
    Get the highest density interval for the distribution, e.g. for a Bayesian posterior, the highest posterior density interval (HPD/HDI)
    Credit: Ying Tong Li (https://yingtongli.me/blog/2022/10/06/scipy-hdi.html) 
    """

    # For a given lower limit, we can compute the corresponding 95% interval
    def interval_width(lower):
        upper = distribution.ppf(distribution.cdf(lower) + level)
        return upper - lower

    # Find such interval which has the smallest width
    # Use equal-tailed interval as initial guess
    initial_guess = distribution.ppf((1-level)/2)
    optimize_result = optimize.minimize(interval_width, initial_guess,method = 'Nelder-Mead')

    lower_limit = optimize_result.x[0]
    width = optimize_result.fun
    upper_limit = lower_limit + width

    return (lower_limit, upper_limit)

distribution = beta(posterior_alpha,posterior_beta)
lower_bound, upper_bound = hdi(distribution,0.95)

plt.plot(x,posterior_dist,color='blue',label="Posterior: Beta(43,9)")
plt.vlines(x=theta_max,ymin=0,ymax=p_max,color='purple',label=f"MAP: θ={round(theta_max,2)}")
plt.fill_between(x,y1=posterior_dist,where=(lower_bound<x)&(x<upper_bound),color='blue',alpha=0.2,label=f"95% HDI: [{round(lower_bound,2)},{round(upper_bound,2)}]")
plt.legend()
plt.show()


## Calculating the Expected Value for a Biased Coin

The expected value of the biased coin can be calculated using the posterior predictive distribution.

In Bayesian statistics, the posterior predictive distribution (PPD) is the distribution of possible unobserved values $\tilde{x}$ conditioned on the set of $n$ previously observed values $\textbf{X}=\{x_1,...,x_n\}$ which are drawn from a distribution that depends on parameter $\theta\in\Theta$ where $\Theta$ is the parameter space.

The PPD is calculated by the marginalization of the distribution of $\tilde{x}$ given $\theta$ over the posterior distribution of $\theta$ given $\textbf{X}$.

$$PPD\stackrel{\text{def}}{=} p(\tilde{x}|\textbf{X})=\int_{\theta}p(\tilde{x}|\theta)p(\theta|\textbf{X})d\theta$$

As we saw in the case of the unbiased coin, $p(\tilde{x}|\theta)$ follows a binomial distribution. Additionally, as we learned above, the conjugate posterior $p(\theta|\textbf{X})$ of the binomial distribution is the beta distribution. Thus,

$$\begin{align}
p(\tilde{x}|\textbf{X}) & = \int_{\theta} p(\tilde{x}|\theta)p(\theta|\textbf{X})d\theta \\
& = \int_0^1Bin(\tilde{x}|n,\theta)Beta(\theta|\alpha,\beta)d\theta \\
& = {n \choose x}\frac{1}{B(\alpha,\beta)}\int_0^1\theta^{x+\alpha-1}(1-\theta)^{n-x+\beta-1}d\theta \\
& = {n \choose x}\frac{B(x+\alpha,n-x+\beta)}{B(\alpha,\beta)}
\end{align}
$$


In [ ]:
from scipy.stats import betabinom

# set parameter values
n = 10 # number of flips
alpha = 43 # number of observed successes + 1
beta = 9 # number of observed failures + 1

#define possible outcomes of success/heads
x_values = np.arange(n+1)

#calculate pmf values
dist = np.array([betabinom.pmf(x,n,alpha,beta) for x in x_values])

#create color masks
win = s_values<=7
lose = s_values>7

#plot the distribution
plt.bar(x_values[win],dist[win],color='green',label='Win $100')
plt.bar(x_values[lose],dist[lose],color='red',label='Lose $100')
plt.ylabel('Probability',fontsize='15')
plt.xlabel('Number of Heads',fontsize='15')
plt.legend()
plt.show()

# calculate expected value of the bet
expected_value = 100*sum(dist[win]) - 100*sum(dist[lose])
print("Expected Value:",f"{round(expected_value,2)}","dollars")

Thus, we would could expect to lose an average of $49.40 every time we make the bet with the magician, given the prior observations of 42 heads in 50 tosses.